# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [72]:
output_data_file = "../output_data/cities.csv"
output_data = pd.read_csv(output_data_file)
output_data.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [73]:
gmaps.configure(api_key=g_key)
locations = output_data[["Lat", "Lng"]].astype(float)
humidity=output_data["Humidity"]

In [74]:
fig = gmaps.figure()
humidity_heat_map = gmaps.heatmap_layer(locations, weights=humidity,
                                        dissipating=False, max_intensity=10,
                                        point_radius=1)
fig.add_layer(humidity_heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [75]:
new_df=output_data.loc[(output_data['Max Temp'] > 65) &(output_data['Max Temp'] < 85) &(output_data['Wind Speed'] < 5) & (output_data['Cloudiness'] == 0)]

new_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [76]:
# Copying and adding Hotel Name Column
hotel_df=new_df.copy()
hotel_df['Hotel Name']=""
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24,
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78,


In [77]:
# Add base url and variables/parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_radius = 5000
params={"radius": hotel_radius,"keyword": "hotel", "type": "lodging",  "key": g_key}

# Create a for loop to iterate through rows
for index, row in hotel_df.iterrows():
    hotel_lat = row['Lat']
    hotel_lng = row['Lng']
    params['location'] = f"{hotel_lat}, {hotel_lng}"
    hotel_resp = requests.get(base_url, params=params).json()
    results = hotel_resp['results']
#     Locating and pulling the first hotel result
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "Not Available"
        pass
    
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24,Goba Lodge
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,Orlanova Boutique Hotel
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Not Available
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mount Mogok Hotel
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78,The Nesbitt Arms
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24,"Grand Hotel des Sablettes Plage, Curio Collect..."


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [79]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
                            info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(humidity_heat_map)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))